In [6]:
using Distributed
using TensorFlow
using Distributions
using Printf

# Generate some synthetic data
x = randn(100, 50)
w = randn(50, 10)
y_prob = exp.(x*w)
y_prob ./= sum(y_prob,dims=2)

function draw(probs)
   y = zeros(size(probs))
   for i in 1:size(probs, 1)
       idx = rand(Categorical(probs[i, :]))
       y[i, idx] = 1
   end
   return y
end

y = draw(y_prob)

100×10 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 ⋮                        ⋮                      
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0 

In [7]:
# Build the model
sess = Session(Graph())

X = placeholder(Float64, shape=[-1, 50])
Y_obs = placeholder(Float64, shape=[-1, 10])

variable_scope("logisitic_model"; initializer=Normal(0, .001)) do
   global W = get_variable("W", [50, 10], Float64)
   global B = get_variable("B", [10], Float64)
end

Y=nn.softmax(X*W + B)

2019-08-20 20:16:42.935650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-08-20 20:16:42.937154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.569
pciBusID: 0000:02:00.0
2019-08-20 20:16:42.937270: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2019-08-20 20:16:42.937325: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2019-08-20 20:16:42.937371: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10
2019-08-20 20:16:42.937419: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuran

<Tensor Softmax:1 shape=(?, 10) dtype=Float64>

In [8]:
 Loss = -reduce_sum(log(Y).*Y_obs)
 optimizer = train.AdamOptimizer()

AdamOptimizer(η=0.001, β1=0.9, β2=0.999, ϵ=1.0e-8)

In [9]:
minimize_op = train.minimize(optimizer, Loss)
saver = train.Saver()

┌ Warning: Your Python TensorFlow client version (1.13.1) is below the TensorFlow backend version (1.14.0). This can cause various errors. Please upgrade your Python TensorFlow installation and then restart Julia.
│ You can upgrade by calling `using Conda; Conda.update();` from Julia.
└ @ TensorFlow /home/dreuter/.julia/packages/TensorFlow/Jif3t/src/version.jl:57


      From worker 10:	/home/dreuter/.julia/conda/3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
      From worker 10:	  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
      From worker 10:	/home/dreuter/.julia/conda/3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
      From worker 10:	  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
      From worker 10:	/home/dreuter/.julia/conda/3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
     

<Saver>

In [10]:
# Run training
 run(sess, global_variables_initializer())
 checkpoint_path = mktempdir()
 @info("Checkpoint files saved in $checkpoint_path")
 for epoch in 1:100
    cur_loss, _ = run(sess, [Loss, minimize_op], Dict(X=>x, Y_obs=>y))
    println(@sprintf("Current loss is %.2f.", cur_loss))
    train.save(saver, sess, joinpath(checkpoint_path, "logistic"), global_step=epoch)
end

┌ Info: Checkpoint files saved in /tmp/tmpjATbBF
└ @ Main In[10]:4
2019-08-20 20:17:45.027450: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10


Current loss is 230.23.
Current loss is 228.76.
Current loss is 227.29.
Current loss is 225.84.
Current loss is 224.39.
Current loss is 222.95.
Current loss is 221.52.
Current loss is 220.09.
Current loss is 218.67.
Current loss is 217.26.
Current loss is 215.86.
Current loss is 214.46.
Current loss is 213.08.
Current loss is 211.70.
Current loss is 210.33.
Current loss is 208.96.
Current loss is 207.61.
Current loss is 206.26.
Current loss is 204.92.
Current loss is 203.59.
Current loss is 202.27.
Current loss is 200.96.
Current loss is 199.65.
Current loss is 198.36.
Current loss is 197.07.
Current loss is 195.79.
Current loss is 194.52.
Current loss is 193.26.
Current loss is 192.00.
Current loss is 190.76.
Current loss is 189.52.
Current loss is 188.29.
Current loss is 187.07.
Current loss is 185.86.
Current loss is 184.66.
Current loss is 183.46.
Current loss is 182.27.
Current loss is 181.10.
Current loss is 179.93.
Current loss is 178.77.
Current loss is 177.61.
Current loss is 

In [34]:
Pkg.status("TensorFlow")

    Status `~/.julia/environments/v1.1/Project.toml`
  [28f6a940] TensorFlow v0.12.0 #master (https://github.com/malmaud/TensorFlow.jl.git)


In [36]:
using PyCall

In [37]:
PyCall.libpython

"/home/dreuter/.julia/conda/3/lib/libpython3.7m.so.1.0"

In [38]:
PyCall.pyprogramname

"/home/dreuter/.julia/conda/3/bin/python"

In [39]:
PyCall.PYTHONHOME

"/home/dreuter/.julia/conda/3:/home/dreuter/.julia/conda/3"

In [11]:
using TensorFlow
include("mnist_loader.jl")

loader = DataLoader()

sess = Session(Graph())

x = placeholder(Float32)
y_ = placeholder(Float32)

W = Variable(zeros(Float32, 784, 10))
b = Variable(zeros(Float32, 10))

run(sess, global_variables_initializer())

2019-08-20 20:18:27.304800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-08-20 20:18:27.305375: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.569
pciBusID: 0000:02:00.0
2019-08-20 20:18:27.305405: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2019-08-20 20:18:27.305416: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2019-08-20 20:18:27.305425: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10
2019-08-20 20:18:27.305433: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuran

In [12]:
 y = nn.softmax(x*W + b)
 cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), axis=[2]))

<Tensor reduce_2:1 shape=unknown dtype=Float32>

In [16]:
 train_step = train.minimize(train.GradientDescentOptimizer(.0001,"gd"), cross_entropy)

<Tensor Group_3:1 shape=() dtype=Any>

In [17]:
correct_prediction = argmax(y, 2) .== argmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float32))

for i in 1:1000
    batch = next_batch(loader, 100)
    println("Batch n:o $(i)")
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
end

Batch n:o 1
Batch n:o 2
Batch n:o 3
Batch n:o 4
Batch n:o 5
Batch n:o 6
Batch n:o 7
Batch n:o 8
Batch n:o 9
Batch n:o 10
Batch n:o 11
Batch n:o 12
Batch n:o 13
Batch n:o 14
Batch n:o 15
Batch n:o 16
Batch n:o 17
Batch n:o 18
Batch n:o 19
Batch n:o 20
Batch n:o 21
Batch n:o 22
Batch n:o 23
Batch n:o 24
Batch n:o 25
Batch n:o 26
Batch n:o 27
Batch n:o 28
Batch n:o 29
Batch n:o 30
Batch n:o 31
Batch n:o 32
Batch n:o 33
Batch n:o 34
Batch n:o 35
Batch n:o 36


InterruptException: InterruptException:

In [ ]:
testx, testy = load_test_set()
println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

In [2]:
using Pkg
Pkg.add("DataStructures")
Pkg.build("DataStructures")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [3]:
using DataStructures
using TensorFlow
using Distributions
using Printf

# Generate some synthetic data
x = randn(100, 50)
w = randn(50, 10)
y_prob = exp.(x*w)
y_prob ./= sum(y_prob,dims=2)

function draw(probs)
    y = zeros(size(probs))
    for i in 1:size(probs, 1)
        idx = rand(Categorical(probs[i, :]))
        y[i, idx] = 1
    end
    return y
end

y = draw(y_prob)

# Build the model
sess = Session(Graph());

X = placeholder(Float64, shape=[-1, 50])
Y_obs = placeholder(Float64, shape=[-1, 10])

variable_scope("logisitic_model"; initializer=Normal(0, .001)) do
    global W = get_variable("W", [50, 10], Float64)
    global B = get_variable("B", [10], Float64)
end

Y=nn.softmax(X*W + B)

Loss = -reduce_sum(log(Y).*Y_obs)
optimizer = train.AdamOptimizer()
minimize_op = train.minimize(optimizer, Loss)
saver = train.Saver()

# Run training
run(sess, global_variables_initializer())
checkpoint_path = mktempdir()
@info("Checkpoint files saved in $checkpoint_path")
for epoch in 1:100
    cur_loss, _ = run(sess, [Loss, minimize_op], Dict(X=>x, Y_obs=>y))
    println(@sprintf("Current loss is %.2f.", cur_loss))
    train.save(saver, sess, joinpath(checkpoint_path, "logistic"), global_step=epoch)
end

2019-08-20 20:35:17.593449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-08-20 20:35:17.593736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.569
pciBusID: 0000:02:00.0
2019-08-20 20:35:17.593759: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2019-08-20 20:35:17.593767: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2019-08-20 20:35:17.593773: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10
2019-08-20 20:35:17.593779: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuran

Current loss is 230.21.
Current loss is 228.76.
Current loss is 227.30.
Current loss is 225.86.
Current loss is 224.43.
Current loss is 223.00.
Current loss is 221.58.
Current loss is 220.16.
Current loss is 218.75.
Current loss is 217.36.
Current loss is 215.97.
Current loss is 214.58.
Current loss is 213.21.
Current loss is 211.84.
Current loss is 210.48.
Current loss is 209.14.
Current loss is 207.79.
Current loss is 206.46.
Current loss is 205.14.
Current loss is 203.82.
Current loss is 202.52.
Current loss is 201.22.
Current loss is 199.93.
Current loss is 198.65.
Current loss is 197.38.
Current loss is 196.11.
Current loss is 194.86.
Current loss is 193.61.
Current loss is 192.38.
Current loss is 191.15.
Current loss is 189.93.
Current loss is 188.72.
Current loss is 187.52.
Current loss is 186.33.
Current loss is 185.15.
Current loss is 183.97.
Current loss is 182.81.
Current loss is 181.65.
Current loss is 180.50.
Current loss is 179.36.
Current loss is 178.23.
Current loss is 

In [4]:
using TensorFlow; const tf = TensorFlow
tf.enable_eager_execution()
m = tf.Sequential()

tf.add(m, tf.Dense(3,10))
tf.add(m, tf.Relu())
tf.add(m, tf.Dense(10, 3))

x=constant(randn(5,3))
y=3x+5
tf.compile(m, optimizer=tf.SGD(lr=1e-4), loss=tf.mse)
tf.fit(m, x, y, n_epochs=200)

2019-08-20 20:36:52.771640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-08-20 20:36:52.772064: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.569
pciBusID: 0000:02:00.0
2019-08-20 20:36:52.772087: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2019-08-20 20:36:52.772095: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2019-08-20 20:36:52.772102: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10
2019-08-20 20:36:52.772109: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuran

In [ ]:
# using TensorFlow
include("mnist_loader.jl")

loader = DataLoader()

sess = Session(Graph())

x = placeholder(Float32)
y_ = placeholder(Float32)

W = Variable(zeros(Float32, 784, 10))
b = Variable(zeros(Float32, 10))

run(sess, global_variables_initializer())

y = nn.softmax(x*W + b)

cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), axis=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

correct_prediction = argmax(y, 2) .== argmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float32))

for i in 1:1000
    batch = next_batch(loader, 100)
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
end

testx, testy = load_test_set()

println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

In [17]:
using DataStructures
using TensorFlow
using Distributions
include("mnist_loader.jl");

loader = DataLoader();

session = Session(Graph());

function weight_variable(shape)
    initial = map(Float32, rand(Normal(0, .001), shape...))
    return Variable(initial)
    end;

function bias_variable(shape)
    initial = fill(Float32(.1), shape...)
    return Variable(initial)
    end;

function conv2d(x, W)
    nn.conv2d(x, W, [1, 1, 1, 1], "SAME")
    end;

function max_pool_2x2(x)
    nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], "SAME")
    end;

2019-08-20 20:51:09.377496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-08-20 20:51:09.378215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.569
pciBusID: 0000:02:00.0
2019-08-20 20:51:09.378258: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.1
2019-08-20 20:51:09.378270: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10
2019-08-20 20:51:09.378278: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10
2019-08-20 20:51:09.378287: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuran

In [18]:
@tf begin

    x = placeholder(Float32)
    y_ = placeholder(Float32)

    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])

    x_image = reshape(x, [-1, 28, 28, 1])

    h_conv1 = nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = nn.relu(h_pool2_flat * W_fc1 + b_fc1)

    keep_prob = placeholder(Float32)
    h_fc1_drop = nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    global y_conv = nn.softmax(h_fc1_drop * W_fc2 + b_fc2)

    global cross_entropy = reduce_mean(-reduce_sum(y_.*log(y_conv), axis=[2]))
end

<Tensor cross_entropy:1 shape=unknown dtype=Float32>

In [19]:
train_step = train.minimize(train.GradientDescentOptimizer(.0001,"gd"), cross_entropy)

correct_prediction = argmax(y_conv, 2) .== argmax(y_, 2)

accuracy = reduce_mean(cast(correct_prediction, Float32))

run(session, global_variables_initializer())

RemoteException: On worker 10:
Python error: PyError ($(Expr(:escape, :(ccall(#= /home/dreuter/.julia/packages/PyCall/ttONZ/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('NodeDef mentions attr \'explicit_paddings\' not in Op<name=Conv2D; signature=input:T, filter:T -> output:T; attr=T:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE]; attr=strides:list(int); attr=use_cudnn_on_gpu:bool,default=true; attr=padding:string,allowed=["SAME", "VALID"]; attr=data_format:string,default="NHWC",allowed=["NHWC", "NCHW"]; attr=dilations:list(int),default=[1, 1, 1, 1]>; NodeDef: {{node Conv2D}}. (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.).')
  File "/home/dreuter/.julia/conda/3/lib/python3.7/site-packages/tensorflow/python/training/saver.py", line 1435, in import_meta_graph
    meta_graph_or_file, clear_devices, import_scope, **kwargs)[0]
  File "/home/d

In [ ]:
for i in 1:200
    batch = next_batch(loader, 50)
    if i%10 == 1
        train_accuracy = run(session, accuracy, Dict(x=>batch[1], y_=>batch[2], keep_prob=>1.0))
        @info("step $i, training accuracy $train_accuracy")
    end
    run(session, train_step, Dict(x=>batch[1], y_=>batch[2], keep_prob=>.5))
end

testx, testy = load_test_set()
test_accuracy = run(session, accuracy, Dict(x=>testx, y_=>testy, keep_prob=>1.0))
@info("test accuracy $test_accuracy")

visualize()